In [1]:
%load_ext autoreload
%autoreload 2
import notebook_setup
from src.config import PROCESSED_DATA_DIR
import os

2025-09-22 21:11:55.679 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: D:\workspace\projects\freelance\Fusion3DNet


Project root added to path: d:\workspace\projects\freelance\Fusion3DNet


In [2]:
from pathlib import Path
FEATURES_DIR = PROCESSED_DATA_DIR / "dataset_129" / 'features'
BREPNET_NPZ_DIR = FEATURES_DIR / "brep"

DINOs14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vits14"
DINOb14_2d_NPZ_DIR = FEATURES_DIR / '2D' / "dinov2_vitb14"
CLIPs16_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_16"
CLIPb32_2d_NPZ_DIR = FEATURES_DIR / '2D' / "ViT-B_32"

DINOs14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vits14"
DINOb14_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "dinov2_vitb14"
CLIPs16_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_16"
CLIPb32_2dv0_NPZ_DIR = FEATURES_DIR / '2D_v0' / "ViT-B_32"

def get_files(step_path: Path, extensions=("stp", "step")):
    return [f for ext in extensions for f in step_path.glob(f"**/*.{ext}")]

brepnet_files = get_files(BREPNET_NPZ_DIR, ('npz',))
print(f"Количество файлов BrepNet: {len(brepnet_files)}")

dino_s14_2d_files = get_files(DINOs14_2d_NPZ_DIR, ('npz',))
dino_b14_2d_files = get_files(DINOb14_2d_NPZ_DIR, ('npz',))
clip_s16_2d_files = get_files(CLIPs16_2d_NPZ_DIR, ('npz',))
clip_b32_2d_files = get_files(CLIPb32_2d_NPZ_DIR, ('npz',))

dino_s14_2dv0_files = get_files(DINOs14_2dv0_NPZ_DIR, ('npz',))
dino_b14_2dv0_files = get_files(DINOb14_2dv0_NPZ_DIR, ('npz',))
clip_s16_2dv0_files = get_files(CLIPs16_2dv0_NPZ_DIR, ('npz',))
clip_b32_2dv0_files = get_files(CLIPb32_2dv0_NPZ_DIR, ('npz',))

print(f"Количество файлов DINO s14 2d: {len(dino_s14_2d_files)}")
print(f"Количество файлов DINO b14 2d: {len(dino_b14_2d_files)}")
print(f"Количество файлов CLIP s16 2d: {len(clip_s16_2d_files)}")
print(f"Количество файлов CLIP b32 2d: {len(clip_b32_2d_files)}")
print(f"Количество файлов DINO s14 2dv0: {len(dino_s14_2dv0_files)}")
print(f"Количество файлов DINO b14 2dv0: {len(dino_b14_2dv0_files)}")
print(f"Количество файлов CLIP s16 2dv0: {len(clip_s16_2dv0_files)}")
print(f"Количество файлов CLIP b32 2dv0: {len(clip_b32_2dv0_files)}")

Количество файлов BrepNet: 129
Количество файлов DINO s14 2d: 129
Количество файлов DINO b14 2d: 129
Количество файлов CLIP s16 2d: 129
Количество файлов CLIP b32 2d: 129
Количество файлов DINO s14 2dv0: 129
Количество файлов DINO b14 2dv0: 129
Количество файлов CLIP s16 2dv0: 129
Количество файлов CLIP b32 2dv0: 129


In [3]:

from src.modeling.vit.metrics import eval_object_max
import numpy as np

def load_npz_features(npz_path):
    data = np.load(npz_path)
    features = data['views']
    return features


model_names = [
    "DINO s14 2d", "DINO b14 2d", "CLIP s16 2d", "CLIP b32 2d",
    "DINO s14 2dv0", "DINO b14 2dv0", "CLIP s16 2dv0", "CLIP b32 2dv0"
]

model_files = [
    dino_s14_2d_files, dino_b14_2d_files, clip_s16_2d_files, clip_b32_2d_files,
    dino_s14_2dv0_files, dino_b14_2dv0_files, clip_s16_2dv0_files, clip_b32_2dv0_files
]

def evaluate_models_for_search(model_names, model_files, load_fn):
    rows = []
    for name, files in zip(model_names, model_files):
        if not files:
            rows.append({"model": name, "queries": 0})
            continue

        m = eval_object_max(files, load_fn)
        m["model"] = name
        rows.append(m)
    return rows

rows = evaluate_models_for_search(model_names, model_files, load_npz_features)

In [ ]:

from src.modeling.BRepNet.metrics import evaluate_brepnet_faces_object_max

out_face_dir = BREPNET_NPZ_DIR / "embeddings"


brep_metrics = evaluate_brepnet_faces_object_max(
        embeddings_dir=out_face_dir,
        normalize=True
    )

In [ ]:
import pandas as pd
df_metrics = pd.DataFrame(rows)
df_brep = pd.DataFrame([{"model": "BRepNet", **brep_metrics}])
# объединяем результаты

df_metrics = pd.concat([df_metrics, df_brep], ignore_index=True)

cols = ["model", "queries", "recall@1", "recall@5", "recall@10", "mAP", "nDCG@5", "nDCG@10",
        "pos_mean", "neg_mean", "margin", "cohens_d"]

df_metrics = df_metrics.reindex(columns=cols)
df_metrics = df_metrics.sort_values([ "margin", "recall@1", "mAP"], ascending=False)
print(df_metrics.to_string(index=False))


# У brenet выключена нормализация (на pos_mean и neg_mean можно не смотреть, так как эвкливово расстояние не нормализовано)
#         model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10      pos_mean  neg_mean   margin  cohens_d
#       BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894 -1.000007e-08 -1.614657 1.614657  1.497281
# DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236  1.000000e+00  0.639031 0.360969  2.803236
#   DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904  1.000000e+00  0.827749 0.172251  2.222084
# DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904  1.000000e+00  0.827749 0.172251  2.222084
#   DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766  1.000000e+00  0.871053 0.128947  2.246197
# CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158  1.000000e+00  0.894322 0.105678  2.684414
# CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314  1.000000e+00  0.906414 0.093586  2.748654
#   CLIP s16 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679  1.000000e+00  0.911103 0.088897  2.551635
#   CLIP b32 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679  1.000000e+00  0.915726 0.084274  2.541060


# У brenet включена нормализация
#         model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10  pos_mean  neg_mean   margin  cohens_d
# DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236       1.0  0.639031 0.360969  2.803236
#   DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
# DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
#   DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766       1.0  0.871053 0.128947  2.246197
#       BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894       1.0  0.876704 0.123296  0.819112
# CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158       1.0  0.894322 0.105678  2.684414
# CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314       1.0  0.906414 0.093586  2.748654
#   CLIP s16 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679       1.0  0.911103 0.088897  2.551635
#   CLIP b32 2d     1032  0.975775  1.000000        1.0 0.987403 0.990679 0.990679       1.0  0.915726 0.084274  2.541060

        model  queries  recall@1  recall@5  recall@10      mAP   nDCG@5  nDCG@10  pos_mean  neg_mean   margin  cohens_d
DINO s14 2dv0     1032  0.897287  0.996124        1.0 0.943849 0.956899 0.958236       1.0  0.639031 0.360969  2.803236
  DINO b14 2d     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
DINO b14 2dv0     1032  0.892442  0.998062        1.0 0.939470 0.954214 0.954904       1.0  0.827749 0.172251  2.222084
  DINO s14 2d     1032  0.891473  0.998062        1.0 0.939244 0.954075 0.954766       1.0  0.871053 0.128947  2.246197
      BRepNet     9895  0.868014  0.999798        1.0 0.927224 0.945822 0.945894       1.0  0.876704 0.123296  0.819112
CLIP b32 2dv0     1032  0.890504  0.996124        1.0 0.939731 0.953821 0.955158       1.0  0.894322 0.105678  2.684414
CLIP s16 2dv0     1032  0.903101  0.996124        1.0 0.946675 0.958978 0.960314       1.0  0.906414 0.093586  2.748654
  CLIP s16 2d     1032  0.975775  1.0000

In [16]:
from src.modeling.vit.searcher import vit_topk
query_index = 0
top = 20
# print(vit_topk("DINO s14 2d",  dino_s14_2d_files,  query=query_index, k=top).to_string())
# vit_topk("DINO b14 2d",  dino_b14_2d_files,  query=query_index, k=top)
# vit_topk("CLIP s16 2d",  clip_s16_2d_files,  query=query_index, k=top)
# vit_topk("CLIP b32 2d",  clip_b32_2d_files,  query=query_index, k=top)

print(vit_topk("DINO s14 2dv0", dino_s14_2dv0_files, query=query_index, k=top).to_string())
# vit_topk("DINO b14 2dv0", dino_b14_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP s16 2dv0", clip_s16_2dv0_files, query=query_index, k=top)
# vit_topk("CLIP b32 2dv0", clip_b32_2dv0_files, query=query_index, k=top)

Поисковая модель: DINO s14 2dv0
                              model     score
0                42. Ejector-01.prt  1.000000
1                42. Ejector-04.prt  0.997702
2                42. Ejector-05.prt  0.996256
3                    42. Ejector-02  0.993263
4                42. Ejector-03.prt  0.990921
5                   42. Ejector.prt  0.986465
6                42. Ejector-10.prt  0.955551
7                42. Ejector-06.prt  0.931798
8                42. Ejector-09.prt  0.923917
9                42. Ejector-07.prt  0.887774
10               42. Ejector-08.prt  0.877828
11  Зацеп трубки направляющий 7.prt  0.817447
12  Зацеп трубки направляющий 2.prt  0.753028
13  Зацеп трубки направляющий 4.prt  0.746652
14  Зацеп трубки направляющий 3.prt  0.742786
15  Зацеп трубки направляющий 1.prt  0.741828
16  Зацеп трубки направляющий 6.prt  0.741007
17  Зацеп трубки направляющий 5.prt  0.735294
18  Зацеп трубки направляющий 8.prt  0.729762
19  Зацеп трубки направляющий 9.prt  0.721372


In [ ]:
from src.modeling.BRepNet.searcher import search_topk
step_files = get_files(PROCESSED_DATA_DIR / "dataset_129" / 'stp')
print(f'Количество файлов STEP: {len(step_files)}')

stp_model = step_files[query_index].stem
print(f'Запрос: {stp_model}')

df, matches, dmin, interval = search_topk(out_face_dir, query_stem=stp_model, k=top, exclude_self=False)

print(df.to_string())


Количество файлов STEP: 129
Запрос: 42. Ejector-01.prt
                       model      score
0         42. Ejector-05.prt   1.193723
1         42. Ejector-04.prt   1.956831
2         42. Ejector-06.prt   2.395920
3            42. Ejector.prt   6.861336
4         42. Ejector-09.prt   7.856229
5         42. Ejector-03.prt   7.998601
6             42. Ejector-02   8.066268
7         42. Ejector-08.prt   8.220020
8         42. Ejector-07.prt   8.463036
9         42. Ejector-10.prt   9.237177
10  44. Extractor Pin-09.prt  10.701600
11  44. Extractor Pin-08.prt  11.137397
12  44. Extractor Pin-10.prt  12.246496
13  44. Extractor Pin-06.prt  13.257885
14  44. Extractor Pin-07.prt  13.269909
15   42. Silencer Fix-04.prt  14.900334
16   42. Silencer Fix-03.prt  15.225912
17   42. Silencer Fix-10.prt  15.510146
18            Защелка 10.prt  15.641165
19  44. Extractor Pin-05.prt  15.683732
